In [1]:
from itertools import product
from tqdm import tqdm

import numpy as np
from keras.datasets import mnist
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.layers import Input
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras import initializers
from keras import backend as K

import matplotlib.pyplot as plt
import cv2

K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [2]:
# Config
np.random.seed(1000)
random_dim = 100

In [3]:
def save_image(imgs, path, row=3, col=3, random=False):
    height, width = imgs.shape[2:]
    output = np.zeros((height * row, width * col), dtype=int)
    for (x, y) in product(range(row), range(col)):
        idx = np.random.randint(0, len(imgs)) if random else row * x + y
        output[height*y:height*(y+1), width*x:width*(x+1)] = imgs[idx, 0]
    cv2.imwrite(path, output)

In [4]:
# load MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train[:, np.newaxis, :, :]
save_image(X_train, path='images/train_data.png', row=10, col=10, random=True)

X_train = (X_train.astype(np.float32) - 127.5) / 127.5
print("X_train.shape: %s" % (X_train.shape,))

X_train.shape: (60000, 1, 28, 28)


In [5]:
# optimizer
adam = Adam(lr=0.0002, beta_1=0.5)

# generator
generator = Sequential()
generator.add(Dense(128 * 7 * 7, input_dim=random_dim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
generator.add(LeakyReLU(0.2))
generator.add(Reshape((128, 7, 7)))
generator.add(UpSampling2D(size=(2, 2)))
generator.add(Conv2D(64, kernel_size=(5, 5), padding='same'))
generator.add(LeakyReLU(0.2))
generator.add(UpSampling2D(size=(2, 2)))
generator.add(Conv2D(1, kernel_size=(5, 5), padding='same', activation='tanh'))
generator.compile(loss='binary_crossentropy', optimizer=adam)

# discriminator
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=(5, 5), strides=(2, 2), padding='same', input_shape=(1, 28, 28), kernel_initializer=initializers.RandomNormal(stddev=0.02)))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=(5, 5), strides=(2, 2), padding='same'))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer=adam)

# combined network
discriminator.trainable = False
gan_input = Input(shape=(random_dim,))
x = generator(gan_input)
gan_output = discriminator(x)
gan = Model(inputs=gan_input, outputs=gan_output)
gan.compile(loss='binary_crossentropy', optimizer=adam)

In [6]:
d_losses = []
g_losses = []

def plot_loss(epoch):
    plt.figure(figsize=(10, 8))
    plt.plot(d_losses, label='Discriminitive loss')
    plt.plot(g_losses, label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('graphs/dcgan_loss_epoch_%05d.png' % epoch)
    plt.close()

In [7]:
def save_model(epoch):
    generator.save('models/dcgan_generator_epoch_%05d.h5' % epoch)
    discriminator.save('models/dcgan_discriminator_epoch_%05d.h5' % epoch)

In [8]:
batch_size = 128
epochs = 200
batch_count = X_train.shape[0] // batch_size

sample_noise = np.random.normal(0, 1, size=[batch_size, random_dim])

steps = 0
for e in range(1, epochs + 1):
    print("--------------- Epoch %d ----------------" % e, flush=True)
    for _ in tqdm(range(batch_count)):
        noise = np.random.normal(0, 1, size=[batch_size, random_dim])
        image_batch = X_train[np.random.randint(0, X_train.shape[0], size=batch_size)]
        
        generated_images = generator.predict(noise)
        X = np.concatenate([image_batch, generated_images])
        
        y_dis = np.zeros(2 * batch_size)
        y_dis[:batch_size] = 0.9
        
        discriminator.trainable = True
        d_loss = discriminator.train_on_batch(X, y_dis)
        
        noise = np.random.normal(0, 1, size=[batch_size, random_dim])
        y_gen = np.ones(batch_size)
        discriminator.trainable = False
        g_loss = gan.train_on_batch(noise, y_gen)
        
        if steps % 10 == 0:
            generated_images = generator.predict(sample_noise)
            generated_images = generated_images * 127.5 + 127.5
            save_image(generated_images, path="images/generated_image_%05d.png" % steps, row=10, col=10, random=False)

        steps += 1
        
    d_losses.append(d_loss)
    g_losses.append(g_loss)

    if e == 1 or e % 5 == 0:
        save_model(e)
    
    # plot_loss(e)

--------------- Epoch 1 ----------------


100%|██████████| 468/468 [00:18<00:00, 25.05it/s]


--------------- Epoch 2 ----------------


100%|██████████| 468/468 [00:17<00:00, 27.10it/s]

--------------- Epoch 3 ----------------


...


100%|██████████| 468/468 [00:17<00:00, 27.38it/s]

--------------- Epoch 199 ----------------



100%|██████████| 468/468 [00:16<00:00, 28.54it/s]

--------------- Epoch 200 ----------------



100%|██████████| 468/468 [00:17<00:00, 27.31it/s]
